<a href="https://colab.research.google.com/github/katemartian/LHoutputsManuscript/blob/master/Fig6_stats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><center>Fig6 stats

This notebook contains statistical analysis of the data from the figure 6 and its supplemental figures.

# Install libraries

Install all necessary libraries to run statistical tests in R.

In [ ]:
# Use R in Jupyter Notebook
%load_ext rpy2.ipython

In [ ]:
%%R
install.packages("readxl")
install.packages("tidyverse")
install.packages("emmeans")
install.packages("maptools", repos = "https://cloud.r-project.org")
install.packages("car", repos = "https://cloud.r-project.org")
install.packages("cowplot", repos = "https://cloud.r-project.org")
install.packages("rstatix", repos = "https://cloud.r-project.org")
install.packages("ggpubr", repos = "https://cloud.r-project.org")

In [ ]:
%%R
library("readxl")
library("tidyverse")
library("emmeans")
library("maptools")
library("car")
library("cowplot")
library("rstatix")
library("ggpubr")

# Directory

Set directory with the source data. Mount disk if you use Google Colab.

In [ ]:
# Mount disk if you use Google Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Change directory
%cd /content/drive/My\ Drive/!lab/Manuscripts/LHoutputs/sourcedata

/content/drive/My Drive/!lab/Manuscripts/LHoutputs/sourcedata


# OFT-TST Fig6 & FigSupp 1


In [ ]:
%%R
df <- read_excel("fig6_LHopto_TST-OFT.xlsx", sheet = "TST-OFT")
df$group = as.factor(df$group)
df$test = as.factor(df$test)
df$output = as.factor(df$output)
df$period = as.factor(df$period)
df$laser = as.factor(df$laser)
df$mobileScore = as.numeric(df$mobileScore)

Outliers

In [ ]:
%%R
df %>%
  group_by(group,test,output,laser) %>%
  identify_outliers(mobileScore)

# A tibble: 9 x 9
  laser group test  output mouse mobileScore period  is.outlier is.extreme
  <fct> <fct> <fct> <fct>  <chr>       <dbl> <fct>   <lgl>      <lgl>     
1 ON    ChR2  OFT   DRN    LHA96       0.321 period1 TRUE       TRUE      
2 ON    ChR2  OFT   DRN    LHA98       0.386 period1 TRUE       FALSE     
3 ON    ChR2  OFT   DRN    LHA94       0.357 period5 TRUE       FALSE     
4 ON    ChR2  OFT   DRN    LHA96       0.356 period5 TRUE       FALSE     
5 ON    ChR2  OFT   VTA    LHA97       0.388 period1 TRUE       FALSE     
6 ON    ChR2  OFT   VTA    LHA94       0.550 period2 TRUE       FALSE     
7 ON    ChR2  OFT   VTA    LHA96       0.364 period5 TRUE       FALSE     
8 ON    ChR2  TST   VTA    LHA98       0.673 period1 TRUE       FALSE     
9 OFF   eYFP  OFT   LHb    LHA40       0.182 period1 TRUE       FALSE     


Normality

In [ ]:
%%R
shapiro <- df %>%
  group_by(group,test,output,laser) %>%
  shapiro_test(mobileScore)
print(shapiro,n=24)

# A tibble: 24 x 7
   laser group test  output variable    statistic       p
   <fct> <fct> <fct> <fct>  <chr>           <dbl>   <dbl>
 1 OFF   ChR2  OFT   DRN    mobileScore     0.951 0.0541 
 2 ON    ChR2  OFT   DRN    mobileScore     0.929 0.00896
 3 OFF   ChR2  OFT   LHb    mobileScore     0.957 0.0974 
 4 ON    ChR2  OFT   LHb    mobileScore     0.975 0.429  
 5 OFF   ChR2  OFT   VTA    mobileScore     0.976 0.458  
 6 ON    ChR2  OFT   VTA    mobileScore     0.976 0.471  
 7 OFF   ChR2  TST   DRN    mobileScore     0.958 0.0999 
 8 ON    ChR2  TST   DRN    mobileScore     0.975 0.424  
 9 OFF   ChR2  TST   LHb    mobileScore     0.951 0.0536 
10 ON    ChR2  TST   LHb    mobileScore     0.966 0.213  
11 OFF   ChR2  TST   VTA    mobileScore     0.969 0.258  
12 ON    ChR2  TST   VTA    mobileScore     0.967 0.234  
13 OFF   eYFP  OFT   DRN    mobileScore     0.950 0.166  
14 ON    eYFP  OFT   DRN    mobileScore     0.987 0.961  
15 OFF   eYFP  OFT   LHb    mobileScore     0.933 0.0

ANOVA

In [ ]:
%%R
res.aov <- anova_test(data=df, dv=mobileScore, wid=mouse, between=group, within=c(test,output,period,laser))
get_anova_table(res.aov)

ANOVA Table (type III tests)

                           Effect  DFn   DFd      F        p p<.05      ges
1                           group 1.00 11.00  4.507 0.057000       0.086000
2                            test 1.00 11.00  3.028 0.110000       0.018000
3                          output 2.00 22.00  2.495 0.106000       0.025000
4                          period 1.84 20.24  1.821 0.189000       0.007000
5                           laser 1.00 11.00 31.043 0.000167     * 0.140000
6                      group:test 1.00 11.00  3.028 0.110000       0.018000
7                    group:output 2.00 22.00  1.994 0.160000       0.020000
8                    group:period 1.84 20.24 11.273 0.000655     * 0.043000
9                     group:laser 1.00 11.00  6.036 0.032000     * 0.031000
10                    test:output 2.00 22.00  0.312 0.735000       0.002000
11                    test:period 2.12 23.36  4.080 0.028000     * 0.007000
12                  output:period 2.77 30.51  2.097 0.1250

In [ ]:
%%R
pwc <- df %>%
  group_by(group,test,output) %>%
  tukey_hsd(mobileScore ~ laser, paired = TRUE, p.adjust.method = "bonferroni") %>%
  select(-term,-null.value,-estimate) 
pwc

# A tibble: 12 x 9
   group test  output group1 group2  conf.low conf.high    p.adj p.adj.signif
   <fct> <fct> <fct>  <chr>  <chr>      <dbl>     <dbl>    <dbl> <chr>       
 1 ChR2  OFT   DRN    OFF    ON      0.0249      0.0853 4.76e- 4 ***         
 2 ChR2  OFT   LHb    OFF    ON      0.0973      0.157  3.57e-10 ****        
 3 ChR2  OFT   VTA    OFF    ON      0.0683      0.124  1.5 e- 9 ****        
 4 ChR2  TST   DRN    OFF    ON      0.0656      0.144  8.23e- 7 ****        
 5 ChR2  TST   LHb    OFF    ON      0.0419      0.120  8.35e- 5 ****        
 6 ChR2  TST   VTA    OFF    ON      0.0954      0.180  5.29e- 9 ****        
 7 eYFP  OFT   DRN    OFF    ON     -0.0167      0.0717 2.18e- 1 ns          
 8 eYFP  OFT   LHb    OFF    ON     -0.00875     0.0683 1.28e- 1 ns          
 9 eYFP  OFT   VTA    OFF    ON      0.000522    0.0985 4.77e- 2 *           
10 eYFP  TST   DRN    OFF    ON     -0.0407      0.0718 5.84e- 1 ns          
11 eYFP  TST   LHb    OFF    ON     -0.0272  